## Project :Thermal Warming 

### DATA EXTRACTION & CLEANING
#### Process

- ** Step 1 : Temperature Data Extraction & Cleaning
    - Temperature raw data is extracted from ftp site ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/v3/csv/
    - Columns not required are dropped and Headers & Column names  are added
    - File is saved for further cleaning process
    
- ** Step 2: CO2 Data Extraction
    - The script extracts the CO2 from files obtained from ftp://ftp.cmdl.noaa.gov/data/greenhouse_gases/co2/flask/surface/
    - Only Monthly files have been considered.

- ** Step 3: Aerosol Data Extraction
    - This script extract data from multiple gzip files, cleans and summarises data. Stores in csv files
    - ftp server is ftp://ftp.cmdl.noaa.gov/aerosol
    
- ** Step 4: Total Solar Irradiance Extraction
    - This script gets data from the SORCE project of NASA for Total Solar Irradiance ("http://lasp.colorado.edu/lisird/latis/nrl2_tsi_P1M.json?time%3E=1970-01-01T12:00&format_time(yyyy-DD-MM))
    - Data is request only after 1970 for each month. 
    - The TSI is represented in watts/ sq. meter

### DATA CLEANING PROCESS
- Drop all Data <1975 and Convert  Averages from ‘str’ to float
- Replace missing Temps (in this case they were ‘-9999’ which would skew data) with np.nan
- In case of Temperature, divide Temps by 100 (the format for the temps was Celsius * 100, probably used to avoid handling decimals)
- Summarize weather stations data to country level on year and month
- Remove countries with missing more than 1 year of temp data, in case of temperature
- Recompile and save the data into csv file

In [160]:
# Import all Dependencies
import os
import csv

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import time
from calendar import monthrange
from pprint import pprint

import json
from pandas.io.json import json_normalize
import requests
import urllib.request as ur
from ftplib import FTP
import ftplib
import gzip

# Change the print display
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

### Setting Global options , variables and filepaths

#### Download URLs, Lookup Data paths

In [161]:
# URLs & FTP Sites for data download
temp_ftp_url = "ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/v3/csv/ghcnm.tavg.v3.3.0.20181219.qcu.dat.csv"
co2_ftp_url = "ftp.cmdl.noaa.gov"
aer_ftp_server = "ftp.cmdl.noaa.gov"
tsi_url = "http://lasp.colorado.edu/lisird/latis/nrl2_tsi_P1M.json?time%3E=1970-01-01T12:00&format_time(yyyy-DD-MM)"

# country code mapping file downloaded from ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/v3/
country_cds = "..\data\_rawdata\LookupFiles\country_codes.csv" 

#Aerosol stations data to be read
stations = ['bnd','brw','mlo','smo','spo','sum','thd']


co2_filepath = "..\data\_rawdata\CO2\surface\MonthlyFiles"
co2_CountryFile = "..\data\_rawdata\LookupFiles\CO2_stations_Country_latest.csv"


#### FILE PATHS - To save output files (images, cleaned data )

In [126]:
## FILE PATHS - to save images, data, output
image_dir = "..\images"
clnd_data_dir = "..\data"
_rawdata_dir = "..\data\_rawdata"

# AEROSOL - Create a local directory with Station Code as the dirname
aer_rawdata_path = "..\data\_rawdata\_aerosol"
if not os.path.exists(aer_rawdata_path):
    os.mkdir(aer_rawdata_path)
    
for nm in stations:
    if not os.path.exists(f"{aer_rawdata_path}\_{nm}"):
        os.mkdir(f"{aer_rawdata_path}\_{nm}")
        print(f"Directory _{nm} created")
    else:    
        print(f"Directory _{nm} already exists")

Directory _bnd already exists
Directory _brw already exists
Directory _mlo already exists
Directory _smo already exists
Directory _spo already exists
Directory _sum already exists
Directory _thd already exists


#### Data filenames to be cleaned

In [70]:
temp_datafile = "..\data\_rawdata\Temp\globalTemp_raw.csv"

### HELPER FUNCTIONS

In [93]:
# FTP Helper function for CO2 file extraction
def FTPCopy(ftpserver = '', login = '', dir = '',fromSubFldr = False, partialMtch = '', copypath = '', copyfilename = ''):
    # check for mandatory variables
    try:
        if(ftpserver == '' or copypath == ''):
            raise ValueError
    except ValueError:
        print("FTPCopy function requires details on FTP server," + \
        "copy path and copy file name. Currently one or more values have not been passed")
        return "Function Error"
    
    print(ftpserver)
    # connect to FTP server
    with FTP(ftpserver) as ftp:
        try:
            ftp.login() ## login into ftp server
            if(dir != ''):
                ftp.cwd(dir) # change dir
            
             #set parent dir
            parent_dir = ftp.pwd()
            
            files_list = [] # empty list to capture files / folders in dir
           
            #get list of files
            ftp.retrlines('NLST', files_list.append)
            
            for f in files_list:
                #search for the file whose filename contains a srch string
                if (partialMtch in f):
                    #open local file to write the chunk data from FTP
                    print(f"{copypath}\_{f}")
                    with open(f"{copypath}\_{f}", 'wb') as copyFile:
                        # Define the callback as a closure so it can access the opened 
                        # file in local scope
                        def callback(data):
                            copyFile.write(data)
                    
                        print(f'RETR %s' % f)
                        ftp.retrbinary('RETR %s' % f, callback) #retrieve file in binary to read and write locally
            
            # close ftp connection
            ftp.quit()
        except ftplib.all_errors as e:
            print('FTP error:', e)
        
    return 'Success'
        

### EXTRACTION PROCESS STARTS....

#### Temperature Data Extraction

In [57]:
# Read temp data from url
req = ur.Request(temp_ftp_url) 
csvfile = ur.urlopen(req)


In [58]:
####################
# Warning
####################
#due to file size over 120M, this step will take over 5 minutes to show result.
df = pd.read_csv(csvfile, header=None, low_memory = False)
print(df)

                 0     1     2     3  4  5  6     7  8  9  ... 41  42    43  \
0       10160355000  1878  TAVG   890        1   950       ...  S   1  1370   
1       10160355000  1879  TAVG  1180        1  1170       ...      1  1450   
2       10160355000  1880  TAVG   960        1  1110       ...      1  1570   
3       10160355000  1931  TAVG -9999            970       ...      1  1550   
4       10160355000  1932  TAVG  1010        1   980       ...        -9999   
5       10160355000  1933  TAVG -9999          -9999       ...      1  1480   
6       10160355000  1934  TAVG  1030        1   970       ...      1  1470   
7       10160355000  1935  TAVG   860        1  1090       ...         1490   
8       10160355000  1936  TAVG  1310        1  1380       ...      1  1410   
9       10160355000  1937  TAVG  1200        1  1250       ...      1  1700   
10      10160355000  1938  TAVG  1060        1   900       ...      1  1560   
11      10160355000  1939  TAVG  1280        1  1180

In [59]:
#drop columns, specify axis=1 for columns and axis=0 for rows
# These columns are flags and not required for further analysis
df.drop(df.columns[[4,5,6,8,9,10,12,13,14,16,17,18,20,21,22,24,25,26,28,29,30,
                    32,33,34,36,37,38,40,41,42,44,45,46,48,49,50]], axis=1, inplace=True)
print(df)

                 0     1     2     3     7     11    15    19    23    27  \
0       10160355000  1878  TAVG   890   950  1110  1610  1980  2240  2490   
1       10160355000  1879  TAVG  1180  1170  1220  1550  1560  2270  2400   
2       10160355000  1880  TAVG   960  1110  1240  1520  1710  2070  2580   
3       10160355000  1931  TAVG -9999   970 -9999 -9999  1850  2390 -9999   
4       10160355000  1932  TAVG  1010   980 -9999  1420  1840 -9999  2290   
5       10160355000  1933  TAVG -9999 -9999 -9999 -9999  1870 -9999 -9999   
6       10160355000  1934  TAVG  1030   970  1190  1550  1760 -9999  2450   
7       10160355000  1935  TAVG   860  1090  1220  1500 -9999  2150 -9999   
8       10160355000  1936  TAVG  1310  1380  1360  1550  1320  1950  2370   
9       10160355000  1937  TAVG  1200  1250  1450  1470 -9999  2200  2320   
10      10160355000  1938  TAVG  1060   900  1100  1320  1680  2160  2390   
11      10160355000  1939  TAVG  1280  1180  1160  1470  1550  2010  2390   

In [60]:
#rename columns
df.columns = ['ID','Year','Type','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
print(df)

                 ID  Year  Type   Jan   Feb   Mar   Apr   May   Jun   Jul  \
0       10160355000  1878  TAVG   890   950  1110  1610  1980  2240  2490   
1       10160355000  1879  TAVG  1180  1170  1220  1550  1560  2270  2400   
2       10160355000  1880  TAVG   960  1110  1240  1520  1710  2070  2580   
3       10160355000  1931  TAVG -9999   970 -9999 -9999  1850  2390 -9999   
4       10160355000  1932  TAVG  1010   980 -9999  1420  1840 -9999  2290   
5       10160355000  1933  TAVG -9999 -9999 -9999 -9999  1870 -9999 -9999   
6       10160355000  1934  TAVG  1030   970  1190  1550  1760 -9999  2450   
7       10160355000  1935  TAVG   860  1090  1220  1500 -9999  2150 -9999   
8       10160355000  1936  TAVG  1310  1380  1360  1550  1320  1950  2370   
9       10160355000  1937  TAVG  1200  1250  1450  1470 -9999  2200  2320   
10      10160355000  1938  TAVG  1060   900  1100  1320  1680  2160  2390   
11      10160355000  1939  TAVG  1280  1180  1160  1470  1550  2010  2390   

In [61]:
#checking each column data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471065 entries, 0 to 471064
Data columns (total 15 columns):
ID      471065 non-null int64
Year    471065 non-null int64
Type    471065 non-null object
Jan     471065 non-null int64
Feb     471065 non-null int64
Mar     471065 non-null int64
Apr     471065 non-null int64
May     471065 non-null int64
Jun     471065 non-null int64
Jul     471065 non-null int64
Aug     471065 non-null int64
Sep     471065 non-null int64
Oct     471065 non-null int64
Nov     471065 non-null int64
Dec     471065 non-null int64
dtypes: int64(14), object(1)
memory usage: 53.9+ MB


In [62]:
# extract country code from ID Column. The first 3 letters represent country code
df['CountryCode'] = df['ID'].apply(lambda i: str(i)[:3])
df['CountryCode'] = df['CountryCode'].astype(int)

df['StationCode'] = df['ID'].apply(lambda i: str(i)[4:])
df['StationCode'] = df['StationCode'].astype(int)
df.tail()


,ID,Year,Type,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,CountryCode,StationCode
471060,80099914001,1967,TAVG,1520,1440,1370,1490,1580,1940,2440,2600,2570,2370,1850,1700,800,9914001
471061,80099914001,1968,TAVG,1430,1230,-9999,1610,1740,1960,2390,2530,2410,2440,1860,1720,800,9914001
471062,80099914001,1969,TAVG,1590,1510,1420,1640,1970,2070,2460,2510,-9999,2150,1830,-9999,800,9914001
471063,80099914001,1970,TAVG,1410,1360,1230,-9999,1830,1970,-9999,2590,2430,2250,1960,1650,800,9914001
471064,80099914001,1971,TAVG,1500,1280,1490,1650,-9999,2060,2320,2320,2450,2210,2000,1700,800,9914001


### Obtain country from country codes lookup 

In [63]:
# Read country names and codes from country_codes.csv file
cntry_cd = pd.read_csv(country_cds, header = None, names = ['CountryCode','Country'])

#change country code to int
cntry_cd['CountryCode'] = cntry_cd['CountryCode'].astype(int)
cntry_cd.head()

,CountryCode,Country
0,101,ALGERIA
1,102,ANGOLA
2,103,BENIN
3,104,BOTSWANA
4,105,BURKINA FASO


### Update temperature raw data with country information

In [64]:
# merge country with temperature dataframe to have country code
temp_withcountry = pd.merge(df, cntry_cd, on = 'CountryCode',how = 'inner')
temp_withcountry.tail()

,ID,Year,Type,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,CountryCode,StationCode,Country
471060,80099914001,1967,TAVG,1520,1440,1370,1490,1580,1940,2440,2600,2570,2370,1850,1700,800,9914001,SHIP STATIONS
471061,80099914001,1968,TAVG,1430,1230,-9999,1610,1740,1960,2390,2530,2410,2440,1860,1720,800,9914001,SHIP STATIONS
471062,80099914001,1969,TAVG,1590,1510,1420,1640,1970,2070,2460,2510,-9999,2150,1830,-9999,800,9914001,SHIP STATIONS
471063,80099914001,1970,TAVG,1410,1360,1230,-9999,1830,1970,-9999,2590,2430,2250,1960,1650,800,9914001,SHIP STATIONS
471064,80099914001,1971,TAVG,1500,1280,1490,1650,-9999,2060,2320,2320,2450,2210,2000,1700,800,9914001,SHIP STATIONS


### Write the Global Raw temperature data to rawdata\Temp folder

In [66]:
#write to csv file
temp_withcountry.to_csv(os.path.join(_rawdata_dir, "Temp\globalTemp_raw.csv"), index = False)

### TEMPERATURE DATA CLEANING

In [71]:
#read csv
avg_temp = pd.read_csv(temp_datafile, low_memory=False)
avg_temp.head()

,ID,Year,Type,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,CountryCode,StationCode,Country
0,10160355000,1878,TAVG,890,950,1110,1610,1980,2240,2490,2680,2320,2680,1370,1150,101,355000,ALGERIA
1,10160355000,1879,TAVG,1180,1170,1220,1550,1560,2270,2400,2510,2240,1750,1450,900,101,355000,ALGERIA
2,10160355000,1880,TAVG,960,1110,1240,1520,1710,2070,2580,2570,2290,2060,1570,1290,101,355000,ALGERIA
3,10160355000,1931,TAVG,-9999,970,-9999,-9999,1850,2390,-9999,2600,2160,1930,1550,1060,101,355000,ALGERIA
4,10160355000,1932,TAVG,1010,980,-9999,1420,1840,-9999,2290,-9999,2440,-9999,-9999,-9999,101,355000,ALGERIA


In [72]:
#convert Year column from str to integer
avg_temp['Year'] = pd.to_numeric(avg_temp['Year'], 'ignore', 'integer')


In [73]:
#drop all years <1970
avg_temp_clean = avg_temp.drop(avg_temp[avg_temp.Year < 1975].index)


In [74]:
#convert temp averages from str to float
avg_temp_clean['Jan'] = pd.to_numeric(avg_temp_clean['Jan'])#, 'ignore', 'float')
avg_temp_clean['Feb'] = pd.to_numeric(avg_temp_clean['Feb'])#, 'ignore', 'float')
avg_temp_clean['Mar'] = pd.to_numeric(avg_temp_clean['Mar'])#, 'ignore', 'float')
avg_temp_clean['Apr'] = pd.to_numeric(avg_temp_clean['Apr'])#, 'ignore', 'float')
avg_temp_clean['May'] = pd.to_numeric(avg_temp_clean['May'])#, 'ignore', 'float')
avg_temp_clean['Jun'] = pd.to_numeric(avg_temp_clean['Jun'])#, 'ignore', 'float')
avg_temp_clean['Jul'] = pd.to_numeric(avg_temp_clean['Jul'])#, 'ignore', 'float')
avg_temp_clean['Aug'] = pd.to_numeric(avg_temp_clean['Aug'])#, 'ignore', 'float')
avg_temp_clean['Sep'] = pd.to_numeric(avg_temp_clean['Sep'])#, 'ignore', 'float')
avg_temp_clean['Oct'] = pd.to_numeric(avg_temp_clean['Oct'])#, 'ignore', 'float')
avg_temp_clean['Nov'] = pd.to_numeric(avg_temp_clean['Nov'])#, 'ignore', 'float')
avg_temp_clean['Dec'] = pd.to_numeric(avg_temp_clean['Dec'])#, 'ignore', 'float')


#drop all values == -9999
#df.replace('-9999', np.nan)
avg_temp_clean = avg_temp_clean.replace(-9999.0, np.nan)


In [75]:
avg_temp_div = avg_temp_clean
avg_temp_div['Jan'] = avg_temp_clean['Jan'].div(100)
avg_temp_div['Feb'] = avg_temp_clean['Feb'].div(100)
avg_temp_div['Mar'] = avg_temp_clean['Mar'].div(100)
avg_temp_div['Apr'] = avg_temp_clean['Apr'].div(100)
avg_temp_div['May'] = avg_temp_clean['May'].div(100)
avg_temp_div['Jun'] = avg_temp_clean['Jun'].div(100)
avg_temp_div['Jul'] = avg_temp_clean['Jul'].div(100)
avg_temp_div['Aug'] = avg_temp_clean['Aug'].div(100)
avg_temp_div['Sep'] = avg_temp_clean['Sep'].div(100)
avg_temp_div['Oct'] = avg_temp_clean['Oct'].div(100)
avg_temp_div['Nov'] = avg_temp_clean['Nov'].div(100)
avg_temp_div['Dec'] = avg_temp_clean['Dec'].div(100)

#avg_temp_div.head()


In [76]:
avg_temp_year = avg_temp_div.groupby(['Country','Year'])[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']].mean()
avg_temp_year_mean = avg_temp_year.fillna(avg_temp_year.mean(axis = 1))
avg_temp_year_mean = np.round(avg_temp_year, decimals=1)
avg_temp_year_mean.head()

Jan  Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct  \
Country     Year                                                             
AFGHANISTAN 1975  1.0  2.5   9.2  15.6  21.8  28.0  30.0  29.0  23.2  15.0   
            1976  4.9  3.6   7.5  15.8  22.9  27.4  29.7  28.6  23.9  16.7   
            1977 -3.4  0.3  12.2  16.4  21.0  27.7   NaN  26.2  20.8  15.8   
            1978 -1.4  2.4   7.6  12.6  21.6  26.6  28.5  26.4  21.0  15.1   
            1979  2.0  4.6   7.5  16.9  17.8  28.1  28.7  25.3  21.6  16.6   

                   Nov  Dec  
Country     Year             
AFGHANISTAN 1975   6.7  4.0  
            1976   6.6  3.3  
            1977  12.6  NaN  
            1978   7.2  6.7  
            1979   6.7  4.0

In [77]:
tempDF = avg_temp_year_mean

In [78]:
#reset index, flattened when exported to CSV, however remained indexed within notebook
tempDF = tempDF.reset_index(['Country','Year'])
tempDF.head()

,Country,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,AFGHANISTAN,1975,1.0,2.5,9.2,15.6,21.8,28.0,30.0,29.0,23.2,15.0,6.7,4.0
1,AFGHANISTAN,1976,4.9,3.6,7.5,15.8,22.9,27.4,29.7,28.6,23.9,16.7,6.6,3.3
2,AFGHANISTAN,1977,-3.4,0.3,12.2,16.4,21.0,27.7,NaN,26.2,20.8,15.8,12.6,NaN
3,AFGHANISTAN,1978,-1.4,2.4,7.6,12.6,21.6,26.6,28.5,26.4,21.0,15.1,7.2,6.7
4,AFGHANISTAN,1979,2.0,4.6,7.5,16.9,17.8,28.1,28.7,25.3,21.6,16.6,6.7,4.0


In [79]:
#tempDF = tempDF.loc[tempDF['Year'] >= 1975, :]
tempDF.count()

Country    8296
Year       8296
Jan        7731
Feb        7751
Mar        7746
Apr        7768
May        7748
Jun        7758
Jul        7736
Aug        7735
Sep        7749
Oct        7731
Nov        7712
Dec        7539
dtype: int64

In [80]:
cntry_year = tempDF[['Country','Year']]
cntry_year['tempVal'] = 1
cntry_year.count()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Country    8296
Year       8296
tempVal    8296
dtype: int64

In [81]:
c_pivot = cntry_year.pivot_table(index = 'Country', columns = 'Year', values = 'tempVal')
c_pivot.reset_index(inplace = True)
c_pivot.head(20)

Year,Country,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFGHANISTAN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,ALBANIA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALGERIA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AMERICAN SAMOA (U.S.A.),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,AMSTERDAM ISLAND (FRANCE),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,ANGOLA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
6,ANTARCTICA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,ARGENTINA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,ARGENTINE BASE IN ANTARCTICA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,ARMENIA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [82]:
print(f"Missing values {c_pivot.isnull().values.sum()}")
cntry_year_clean = c_pivot.dropna(how = 'any')
print(f"Missing values {c_pivot.isnull().values.sum()}")
print(f"Count of Countries after removing NA : {cntry_year_clean['Country'].count()}")
cntry_year_clean.head()

Missing values 1560
Missing values 1560
Count of Countries after removing NA : 120


Year,Country,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2,ALGERIA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AMERICAN SAMOA (U.S.A.),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,ANTARCTICA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,ARGENTINA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,ARGENTINE BASE IN ANTARCTICA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [83]:
cntryList = list(cntry_year_clean['Country'])
len(cntryList)

120

In [84]:
AvgTemp_fnl = tempDF.loc[tempDF['Country'].isin(cntryList), :]

In [86]:
print(f"Unique Countries : {AvgTemp_fnl['Country'].nunique()}")
AvgTemp_fnl.head()

Unique Countries : 120


,Country,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
41,ALGERIA,1975,8.9,10.3,12.1,16.4,20.7,23.8,28.5,27.8,24.7,18.9,12.8,10.9
42,ALGERIA,1976,8.6,10.6,11.3,15.8,19.7,24.9,27.5,25.0,24.2,19.0,11.7,11.1
43,ALGERIA,1977,11.0,12.4,14.8,16.9,20.0,23.0,28.7,26.0,24.0,19.4,14.0,11.7
44,ALGERIA,1978,9.1,13.1,13.8,16.9,20.0,25.1,28.1,27.8,24.8,17.8,12.6,12.9
45,ALGERIA,1979,13.0,12.1,14.4,15.0,21.0,26.5,29.3,29.2,23.8,20.0,11.5,10.2


In [88]:
AvgTemp_fnl = AvgTemp_fnl.interpolate(method='linear', axis=0).ffill().bfill()
AvgTemp_fnl.head(10)

,Country,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
41,ALGERIA,1975,8.9,10.3,12.1,16.4,20.7,23.8,28.5,27.8,24.7,18.9,12.8,10.9
42,ALGERIA,1976,8.6,10.6,11.3,15.8,19.7,24.9,27.5,25.0,24.2,19.0,11.7,11.1
43,ALGERIA,1977,11.0,12.4,14.8,16.9,20.0,23.0,28.7,26.0,24.0,19.4,14.0,11.7
44,ALGERIA,1978,9.1,13.1,13.8,16.9,20.0,25.1,28.1,27.8,24.8,17.8,12.6,12.9
45,ALGERIA,1979,13.0,12.1,14.4,15.0,21.0,26.5,29.3,29.2,23.8,20.0,11.5,10.2
46,ALGERIA,1980,9.2,11.4,14.1,16.2,19.7,26.2,27.8,29.2,25.3,18.6,14.6,7.5
47,ALGERIA,1981,7.2,10.3,16.1,18.6,21.8,26.3,27.4,27.5,25.2,21.1,13.8,12.1
48,ALGERIA,1982,9.6,10.3,12.6,15.9,21.1,26.0,29.7,28.5,24.9,19.2,13.2,8.5
49,ALGERIA,1983,7.5,10.5,13.6,18.4,22.4,25.6,29.2,29.0,25.1,20.3,16.2,10.2
50,ALGERIA,1984,9.0,9.9,13.0,18.6,20.7,26.1,28.3,27.8,24.4,17.6,14.5,10.1


In [89]:
AvgTemp_fnl.to_csv (os.path.join(clnd_data_dir, "NaN_Replaced_Temps.csv"), index = None, header=True)

### ****************** Temperature data cleaned ************************

### CO2 Data Extraction & Cleaning

In [94]:
####################################
# WARNING : Function will take 3 mins to complete
####################################
# Call to helper function FTPCopy to extract files from FTP server to local machine
FTPCopy(ftpserver = co2_ftp_url, dir = 'data/greenhouse_gases/co2/flask/surface', partialMtch = "_month", \
        copypath = co2_filepath)

ftp.cmdl.noaa.gov
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_abp_surface-flask_1_ccgg_month.txt
RETR co2_abp_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_alt_surface-flask_1_ccgg_month.txt
RETR co2_alt_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_ams_surface-flask_1_ccgg_month.txt
RETR co2_ams_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_amy_surface-flask_1_ccgg_month.txt
RETR co2_amy_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_asc_surface-flask_1_ccgg_month.txt
RETR co2_asc_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_ask_surface-flask_1_ccgg_month.txt
RETR co2_ask_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_avi_surface-flask_1_ccgg_month.txt
RETR co2_avi_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_azr_surface-flask_1_ccgg_month.txt
RETR co2_azr_surfa

..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_pocs35_surface-flask_1_ccgg_month.txt
RETR co2_pocs35_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_psa_surface-flask_1_ccgg_month.txt
RETR co2_psa_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_pta_surface-flask_1_ccgg_month.txt
RETR co2_pta_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_rpb_surface-flask_1_ccgg_month.txt
RETR co2_rpb_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_scsn03_surface-flask_1_ccgg_month.txt
RETR co2_scsn03_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_scsn06_surface-flask_1_ccgg_month.txt
RETR co2_scsn06_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_scsn09_surface-flask_1_ccgg_month.txt
RETR co2_scsn09_surface-flask_1_ccgg_month.txt
..\data\_rawdata\CO2\surface\MonthlyFiles\_co2_scsn12_surface-flask_1_ccgg_month.txt
RETR co2_

'Success'

In [97]:
################### Obtain the num of files to be read #######################################
fileList = []
# Read the dir and list the name of the files
for root, dirs, files in os.walk(co2_filepath):
    if(files):
        for name in files:
            if(name not in fileList):
                fileList.append(name)

        print(f"Data has to be extracted from {len(fileList)} files")
    else:
        print("No files present to extract data. Copy files from FTP to proceed further")

Data has been extracted from 95 files


### Read data from each CO2 monthly files and load it into a dataframe

In [101]:
txtData = []
CO2Data = pd.DataFrame()

for i, f in enumerate(fileList):
    with(open(f"{co2_filepath}\{f}", "r")) as txtFile:
        txtData = txtFile.read().splitlines()
    
    # DAta is space delimited format. But not uniform. 
    # Change spaces to , and split field values by commas
    tmpList = [l.replace(" ",",") for l in txtData[int(txtData[0].rsplit(" ", 1)[1]):]]
    tmpList = [l.split(",") for l in tmpList]
    
    # add data to dataframe
    CO2Data = CO2Data.append(tmpList)



#Format the values and correct incorrect split of data    
CO2Data['month'] = CO2Data[2].add(CO2Data[3], fill_value = 0)
CO2Data['month'] = CO2Data['month'].add(CO2Data[4], fill_value = 0)
CO2Data[[5,6]] = CO2Data[[5,6]].apply(pd.to_numeric)
CO2Data['CO2Value'] = CO2Data[5].add(CO2Data[6], fill_value = 0)

# Drop cols not needed
CO2Data = CO2Data.drop([2,3,4,5,6], axis = 1)

# Change column names, and data types
CO2Data.rename(columns = {0 : 'Code',1 : 'year'}, inplace = True)

#convert all number columns to numeric datatype
CO2Data[['CO2Value','year','month']] = CO2Data[['CO2Value','year','month']].apply(pd.to_numeric)


CO2Data.head()

,Code,year,month,CO2Value
0,ABP,2006,10,380.72
1,ABP,2006,11,380.82
2,ABP,2006,12,380.92
3,ABP,2007,1,381.02
4,ABP,2007,2,381.09


In [103]:
# Remove all data before 1970.
CO2Data = CO2Data.loc[CO2Data['year']>= 1975,:]

print(f"data from {CO2Data['year'].min()} to {CO2Data['year'].max()}")

data from 1975 to 2017


In [104]:
#Reshape data to be same as Temp view
CO2Data_fnl = CO2Data.pivot_table('CO2Value', ['Code','year'], 'month')

CO2Data_fnl = CO2Data_fnl.reset_index()


CO2Data_fnl.rename_axis("", axis = 1, inplace = True)

mnth_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov', 12:'Dec'}
CO2Data_fnl.rename(columns = mnth_names, inplace = True)

CO2Data_fnl.head()


,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,000,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.66,NaN,NaN,NaN
1,000,1988,351.31,352.37,353.04,353.35,353.30,353.05,352.68,352.29,NaN,NaN,NaN,NaN
2,000,1989,354.37,355.10,355.46,355.34,354.93,354.38,353.86,353.47,353.19,353.17,353.46,354.11
3,000,1990,354.88,355.47,355.86,356.00,355.96,355.76,355.39,354.85,354.32,354.08,354.28,354.94
4,000,1991,355.71,356.40,356.87,357.04,356.94,356.64,356.18,355.51,354.88,354.61,354.82,355.46


In [107]:
#Fix Missing Values - with avgs of NAN row
CO2Data_clean = CO2Data_fnl

rowMean = CO2Data_clean[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov', 'Dec']].mean(axis = 1)


CO2Data_clean = CO2Data_clean.apply(lambda r: r.fillna(rowMean[r.index]))

CO2Data_clean = CO2Data_clean.round(decimals = 2)

#Check if NAN exists
printmd(f"#### Num of duplicates is {CO2Data_clean.isnull().values.sum()}")

CO2Data_clean.head()

#### Num of duplicates is 0

,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,000,1987,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66
1,000,1988,351.31,352.37,353.04,353.35,353.30,353.05,352.68,352.29,352.67,352.67,352.67,352.67
2,000,1989,354.37,355.10,355.46,355.34,354.93,354.38,353.86,353.47,353.19,353.17,353.46,354.11
3,000,1990,354.88,355.47,355.86,356.00,355.96,355.76,355.39,354.85,354.32,354.08,354.28,354.94
4,000,1991,355.71,356.40,356.87,357.04,356.94,356.64,356.18,355.51,354.88,354.61,354.82,355.46


In [108]:
# Lookup Country and Add it to CO2 Data
ctry_stat = pd.read_csv(co2_CountryFile)
ctry_stat.head()


,Code,Name,Country,Latitude,Longitude,Elevation (meters),Time from GMT
0,AAO,"Airborne Aerosol Observatory, Bondville, Illinois",United States,40.050,-88.370,230.0,-6 hours
1,ABP,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
2,ABQ,"Albuquerque, New Mexico",United States,35.038,-106.622,1617.0,-7 hours
3,ACG,Alaska Coast Guard,United States,65.000,-165.000,0.0,-8 hours
4,ALT,"Alert, Nunavut",Canada,82.451,-62.507,190.0,-4 hours


In [109]:
# Merge on Station Code for details about the station
CO2Data_Full = pd.merge(CO2Data_clean, ctry_stat, how = 'inner', on = 'Code')
CO2Data_Full.head() 

,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Name,Country,Latitude,Longitude,Elevation (meters),Time from GMT
0,ABP,2006,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.72,380.82,380.92,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
1,ABP,2007,381.02,381.09,380.99,380.89,381.15,381.82,382.26,382.19,382.34,381.53,381.53,381.53,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
2,ABP,2008,384.23,384.23,383.02,382.64,382.83,383.83,384.34,384.20,384.77,385.46,385.64,385.56,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
3,ABP,2009,384.85,385.28,386.19,385.20,384.39,385.18,385.67,386.08,386.20,386.18,386.59,386.81,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
4,ALT,1985,344.06,344.06,344.06,344.06,344.06,349.94,343.98,337.96,339.09,342.85,345.66,348.97,"Alert, Nunavut",Canada,82.451,-62.507,190.0,-4 hours


In [110]:
# Roll up stations to Country

CO2Data_Cntry_GDF = CO2Data_Full[['Code','year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep',\
                                  'Oct','Nov','Dec','Country']].groupby(['year','Country']).agg(np.mean)

CO2Data_Cntry_GDF.reset_index(inplace= True)

CO2Data_Cntry_GDF = CO2Data_Cntry_GDF.round(decimals = 2)

CO2Data_Cntry_GDF.head()


,year,Country,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1975,American Samoa,330.65,330.65,330.20,329.63,329.66,330.60,330.83,330.91,331.79,331.64,330.88,330.56
1,1975,United States,333.32,333.24,333.22,333.30,333.16,332.18,329.34,327.23,327.44,329.78,331.89,333.15
2,1976,American Samoa,330.52,330.60,330.82,331.17,331.42,331.10,330.98,331.53,331.90,332.17,332.62,332.95
3,1976,United States,332.56,332.80,333.28,334.17,334.15,332.84,331.08,329.01,328.64,329.98,331.74,333.10
4,1977,American Samoa,332.98,332.48,332.21,332.66,332.82,332.56,332.81,332.91,332.62,332.57,333.01,333.68


In [111]:
# Roll up stations to Global level

CO2Data_glb_GDF = CO2Data_Full[['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep',\
                                  'Oct','Nov','Dec']].groupby(['year']).agg(np.mean)

CO2Data_glb_GDF.reset_index(inplace= True)

CO2Data_glb_GDF = CO2Data_glb_GDF.round(decimals = 2)

CO2Data_glb_GDF.head()

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1975,332.43,332.38,332.21,332.08,331.99,331.65,329.84,328.45,328.89,330.40,331.55,332.29
1,1976,332.26,332.49,332.93,333.74,333.76,332.59,331.06,329.37,329.10,330.30,331.87,333.08
2,1977,333.94,334.30,334.73,335.59,335.93,334.96,333.34,331.67,330.96,331.78,333.53,334.95
3,1978,335.66,336.13,336.54,336.88,337.07,336.66,334.92,333.05,332.72,333.68,335.20,336.13
4,1979,337.40,337.90,338.65,339.11,339.25,338.33,336.09,334.40,334.50,335.80,337.08,337.80


In [112]:
#Rename Columns to Standard maintained by other files
CO2Data_Full.rename(columns = {'year':'Year'}, inplace = True)
CO2Data_Cntry_GDF.rename(columns = {'year':'Year'}, inplace = True)
CO2Data_glb_GDF.rename(columns = {'year':'Year'}, inplace = True)

In [113]:
#Output to csv folder
CO2Data_Full.to_csv("..\data\CO2_RawData.csv", index = False)  ## Entire Raw DAta with stations

CO2Data_Cntry_GDF.to_csv("..\data\CO2_BYCountry.csv", index = False) ## Grouped by Country

CO2Data_glb_GDF.to_csv("..\data\CO2_GlobalSummary.csv", index = False) ## Values rolled up to global level

### ****************** CO2 data cleaned ************************

### AEROSOL DATA Extraction and Cleaning

#### Copy files from AEROSOL FTP Server

########################################
######## WARNING : COPYING FILES WILL TAKE 5 mins or more
########################################

In [123]:
for s in stations:
    # Open FTP Server and read the folder
    with FTP(aer_ftp_server) as ftp:
        try:
            ftp.login() ## login into ftp server
            ftp.cwd("aerosol/"+s) # change directory to the relevant station's dir

            #set parent dir
            parent_dir = ftp.pwd() # make sure parent directory is saved

            years = [] # list to capture the num of years records are available

            #get list of files
            ftp.retrlines('NLST', years.append)

            #browse through each year to obtain the .gzip raw data for particle aerosol concentration
            for y in years:
                ftp.cwd(y) # go to specific year dir
                print(ftp.pwd()) # print the current directory

                files = [] #get the list of files from the dir

                ftp.retrlines('NLST', files.append)

                #search for the file specific to particle concentration and ignore others
                filenm = ""
                filenm = [f for f in files if "particle_number_concentration" in f]

                print(filenm) #print filenm - empty if not present
                
                #if the filenm is present then copy to local drive
                if( filenm):
                    #open local gzip file to write the chunk data from FTP
                    with open(f"..\data\_rawdata\_aerosol\_{s}\{y}{s}_conc.gz", 'wb') as f:
                        # Define the callback as a closure so it can access the opened 
                        # file in local scope
                        def callback(data):
                            f.write(data)
                
                        ftp.retrbinary('RETR %s' % filenm[0], callback) #retrieve file in binary to read and write locally
                
                #change to parent dir to access next year
                ftp.cwd(parent_dir)
            
            # close ftp connection
            ftp.quit()

        except ftplib.all_errors as e:
            print('FTP error:', e)

printmd("### Successfully copied all files locally")

/data/aer/bnd/1994
['US0035R.19940714000000.20171019222346.particle_number_concentration.aerosol.171d.1h.lev2.nas.gz']
/data/aer/bnd/1995
['US0035R.19950101000000.20171019222411.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/bnd/1996
['US0035R.19960101000000.20171019222445.particle_number_concentration.aerosol.366d.1h.lev2.nas.gz']
/data/aer/bnd/1997
['US0035R.19970101000000.20171019222535.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/bnd/1998
['US0035R.19980101000000.20171019222649.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/bnd/1999
['US0035R.19990101000000.20171019222758.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/bnd/2000
['US0035R.20000101000000.20171019222910.particle_number_concentration.aerosol.366d.1h.lev2.nas.gz']
/data/aer/bnd/2001
['US0035R.20010101000000.20171019223019.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/bnd/2002
['US0035R.20020101000000.20171019223140

/data/aer/brw/2008
['US0008R.20080101000000.20171019231630.particle_number_concentration.aerosol.70d.1h.lev2.nas.gz', 'US0008R.20080311000000.20171019231630.particle_number_concentration.aerosol.296d.1h.lev2.nas.gz']
/data/aer/brw/2009
['US0008R.20090101000000.20171019231739.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/brw/2010
['US0008R.20100101000000.20171019231852.particle_number_concentration.aerosol.95d.1h.lev2.nas.gz', 'US0008R.20100406000000.20171019231852.particle_number_concentration.aerosol.270d.1h.lev2.nas.gz']
/data/aer/brw/2011
['US0008R.20110101000000.20171019232003.particle_number_concentration.aerosol.37d.1h.lev2.nas.gz', 'US0008R.20110207000000.20171019232003.particle_number_concentration.aerosol.4914h.1h.lev2.nas.gz', 'US0008R.20110830180000.20171019232003.particle_number_concentration.aerosol.2958h.1h.lev2.nas.gz']
/data/aer/brw/2012
['US0008R.20120101000000.20171019232122.particle_number_concentration.aerosol.366d.1h.lev2.nas.gz']
/data/aer/br

['US6001R.19780101000000.20171020000922.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1979
['US6001R.19790101000000.20171020000940.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1980
['US6001R.19800101000000.20171020000956.particle_number_concentration.aerosol.366d.1h.lev2.nas.gz']
/data/aer/smo/1981
['US6001R.19810101000000.20171020001008.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1982
['US6001R.19820101000000.20171020001029.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1983
['US6001R.19830101000000.20171020001050.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1984
['US6001R.19840101000000.20171020001112.particle_number_concentration.aerosol.366d.1h.lev2.nas.gz']
/data/aer/smo/1985
['US6001R.19850101000000.20171020001132.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/smo/1986
['US6001R.19860101000000.20171020001153.particle_number_conc

['US6005G.20140101000000.20171020010410.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/thd/2015
['US6005G.20150101000000.20171020010533.particle_number_concentration.aerosol.1y.1h.lev2.nas.gz']
/data/aer/thd/2016
['US6005G.20160101000000.20171020010653.particle_number_concentration.aerosol.8d.1h.lev2.nas.gz', 'US6005G.20160109000000.20171020010653.particle_number_concentration.aerosol.31d.1h.lev2.nas.gz', 'US6005G.20160209000000.20171020010653.particle_number_concentration.aerosol.263h.1h.lev2.nas.gz', 'US6005G.20160219230000.20171020010653.particle_number_concentration.aerosol.7585h.1h.lev2.nas.gz']
/data/aer/thd/2017
['US6005G.20170101000000.20180218054702.particle_number_concentration.aerosol.3644h.1h.lev2.nas.gz']


### Extract Aerosol data from .nas files

In [143]:
# Initialise dataframe and any declarations

getItems = ['Station GAW-ID:', 'Station state/province:','Measurement latitude:','Measurement longitude:']

monthly_avg_AER = pd.DataFrame(columns = ['StationID', 'Location','latitude','longitude',\
                                         'Year','Month', 'Avg.Conc'])


In [144]:
# get the list of stations to loop through
aer_stations = []
for roots,dirs,files in os.walk(aer_rawdata_path):
    if(dirs):
        aer_stations = dirs

aer_stations

['_bnd', '_brw', '_mlo', '_smo', '_spo', '_sum', '_thd']

#### Read from 161 folders and combine the data into a dataframe

In [145]:

#Loop through stations to extract data
for s in aer_stations:
    print(f"Extracting and Loading data for station - {s} ")
    
    for root, dirs, files in os.walk(os.path.join(aer_rawdata_path,s)):
        for name in files:
            if(name.rsplit('.',1)[1] == 'gz'):

                file_content = []
                #open gz file in rt mode - opens in read & text mode
                with gzip.open(os.path.join(root,name), 'rt') as f:
                    file_content = f.read().splitlines()

                if(file_content):
                    ### Collect Station Details
                    Station_Info = {'Station GAW-ID:':['None'], 'Station state/province:':['None'],\
                                    'Measurement latitude:':['None'],'Measurement longitude:':['None']}          

                    for itm in getItems:
                        Station_Info[itm] = [file_content[i].rsplit(" ",1)[1] for i, s in enumerate(file_content) if itm in s]

                    for key,val in Station_Info.items():
                        if(not Station_Info[key]):
                            Station_Info[key] = ['None']
                        
                                       
                    ### Collect Data
                    dataStrt = int(file_content[0].split(" ")[0])
                    yr =  int(file_content[6].split(" ")[0])
                    cols = [v for v in file_content[dataStrt-1].split(" ") if v != ''] 

                    station_data  = []

                    for i, row in enumerate(file_content[dataStrt:]):
                        station_data.append([v for v in row.split(" ") if v != ''])

                    
                    #temp dataframe to store the station data
                    df = pd.DataFrame(station_data, columns = cols)

                                           
                    ### CLean Data
                    #drop cols not required
                    df = df[['start_time','end_time','st_y','ed_y','conc']]

                    df = df.apply(pd.to_numeric)

                    df = df.replace(99999.9, value = np.nan)

                    #Get num of days per year
                    days_in_mnth = []

                    for i in range(1,13,1):
                        days_in_mnth.append(monthrange(yr, i)[1]*24)

                    days_in_mnth = np.cumsum(days_in_mnth)

                    ### Get Monthly avg
                    df_monthly_avg = pd.DataFrame(columns = ['StationID', 'Location','latitude','longitude',\
                                                 'Year','Month', 'Avg.Conc'])
                    strt = 0

                    for i, val in enumerate(days_in_mnth):
                        df_monthly_avg.loc[i] = [Station_Info['Station GAW-ID:'][0], Station_Info['Station state/province:'][0],\
                                                 Station_Info['Measurement latitude:'][0],Station_Info['Measurement longitude:'][0],\
                                                 yr, i+1, df[strt:val]['conc'].mean()]
                        strt = val+1

                    ## append with master DF
                    monthly_avg_AER = monthly_avg_AER.append(df_monthly_avg, sort=False)

print("Extraction done and Data Load to dataframe")
print(f"Total data loaded is {len(monthly_avg_AER)}")

Extracting and Loading data for station - _bnd 
Extracting and Loading data for station - _brw 
Extracting and Loading data for station - _mlo 
Extracting and Loading data for station - _smo 
Extracting and Loading data for station - _spo 
Extracting and Loading data for station - _sum 
Extracting and Loading data for station - _thd 
Extraction done and Data Load to dataframe
Total data loaded is 1944


In [146]:
monthly_avg_AER.head()

,StationID,Location,latitude,longitude,Year,Month,Avg.Conc
0,BND,Illinois,40.05000,-88.36667,1994,1,7416.095385
1,BND,Illinois,40.05000,-88.36667,1994,2,8250.276304
2,BND,Illinois,40.05000,-88.36667,1994,3,7657.565079
3,BND,Illinois,40.05000,-88.36667,1994,4,NaN
4,BND,Illinois,40.05000,-88.36667,1994,5,NaN


In [150]:
# Collate only data upto 2017
monthly_avg_AER = monthly_avg_AER[(monthly_avg_AER['Year'] >= 1975) & (monthly_avg_AER['Year'] <= 2017)]

print(f"Data starts from year {monthly_avg_AER['Year'].min()} and ends in {monthly_avg_AER['Year'].max()}")

Data starts from year 1975 and ends in 2017


In [151]:
# Reshape Data with Month along the column instead of rows
monthly_avg_AER = monthly_avg_AER.pivot_table(index = ['StationID', 'Location','latitude','longitude','Year'], \
                           columns = 'Month', values = 'Avg.Conc')

monthly_avg_AER.reset_index(inplace = True)

monthly_avg_AER.head()

Month,StationID,Location,latitude,longitude,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,BND,Illinois,40.05000,-88.36667,1994,7416.095385,8250.276304,7657.565079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BND,Illinois,40.05000,-88.36667,1995,5968.775814,7162.291803,8252.129868,6663.806811,6226.199859,4933.423659,6183.971968,4861.151709,8760.603648,6857.581215,5655.194150,6026.326203
2,BND,Illinois,40.05000,-88.36667,1996,NaN,NaN,NaN,1100.308333,3446.747376,3127.360201,4073.380619,6261.032070,6619.884828,6095.886916,NaN,3659.170427
3,BND,Illinois,40.05000,-88.36667,1997,5061.243383,3513.422356,4041.113459,2966.039888,2012.236145,NaN,3886.328283,5925.260870,7167.643339,9868.816059,5873.094715,4452.612195
4,BND,Illinois,40.05000,-88.36667,1998,4977.414286,2695.786942,1385.488854,2085.668524,3734.474488,7184.377101,6224.556716,5529.677486,7570.577287,6596.904367,5956.719236,3211.306470


In [152]:

# Change col names to month names
mnth_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov', 12:'Dec'}

monthly_avg_AER.rename(columns = mnth_names, inplace = True)
monthly_avg_AER.rename_axis("", axis = 1, inplace = True)

monthly_avg_AER.head()

,StationID,Location,latitude,longitude,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,BND,Illinois,40.05000,-88.36667,1994,7416.095385,8250.276304,7657.565079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BND,Illinois,40.05000,-88.36667,1995,5968.775814,7162.291803,8252.129868,6663.806811,6226.199859,4933.423659,6183.971968,4861.151709,8760.603648,6857.581215,5655.194150,6026.326203
2,BND,Illinois,40.05000,-88.36667,1996,NaN,NaN,NaN,1100.308333,3446.747376,3127.360201,4073.380619,6261.032070,6619.884828,6095.886916,NaN,3659.170427
3,BND,Illinois,40.05000,-88.36667,1997,5061.243383,3513.422356,4041.113459,2966.039888,2012.236145,NaN,3886.328283,5925.260870,7167.643339,9868.816059,5873.094715,4452.612195
4,BND,Illinois,40.05000,-88.36667,1998,4977.414286,2695.786942,1385.488854,2085.668524,3734.474488,7184.377101,6224.556716,5529.677486,7570.577287,6596.904367,5956.719236,3211.306470


In [153]:
# convert Concentration values to numeric and round off at 2nd decimal place
monthly_avg_AER.iloc[:, 2:].apply(pd.to_numeric)
monthly_avg_AER = monthly_avg_AER.round(decimals = 2)

monthly_avg_AER.head()

,StationID,Location,latitude,longitude,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,BND,Illinois,40.05000,-88.36667,1994,7416.10,8250.28,7657.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BND,Illinois,40.05000,-88.36667,1995,5968.78,7162.29,8252.13,6663.81,6226.20,4933.42,6183.97,4861.15,8760.60,6857.58,5655.19,6026.33
2,BND,Illinois,40.05000,-88.36667,1996,NaN,NaN,NaN,1100.31,3446.75,3127.36,4073.38,6261.03,6619.88,6095.89,NaN,3659.17
3,BND,Illinois,40.05000,-88.36667,1997,5061.24,3513.42,4041.11,2966.04,2012.24,NaN,3886.33,5925.26,7167.64,9868.82,5873.09,4452.61
4,BND,Illinois,40.05000,-88.36667,1998,4977.41,2695.79,1385.49,2085.67,3734.47,7184.38,6224.56,5529.68,7570.58,6596.90,5956.72,3211.31


In [156]:
#Obtain row-wise avg to replace NAN
rowAvg = monthly_avg_AER[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']].mean(axis = 1)
rowAvg

# Replace NAN with row-wise mean
monthly_avg_AER = monthly_avg_AER.apply(lambda r: r.fillna(rowAvg[r.index]))
monthly_avg_AER = monthly_avg_AER.round(decimals = 2)

#Check if NAN exists
printmd(f"### There are {monthly_avg_AER.isnull().values.sum()} null values")

monthly_avg_AER.head()


### There are 0 null values

,StationID,Location,latitude,longitude,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,BND,Illinois,40.05000,-88.36667,1994,7416.10,8250.28,7657.57,7774.65,7774.65,7774.65,7774.65,7774.65,7774.65,7774.65,7774.65,7774.65
1,BND,Illinois,40.05000,-88.36667,1995,5968.78,7162.29,8252.13,6663.81,6226.20,4933.42,6183.97,4861.15,8760.60,6857.58,5655.19,6026.33
2,BND,Illinois,40.05000,-88.36667,1996,4297.97,4297.97,4297.97,1100.31,3446.75,3127.36,4073.38,6261.03,6619.88,6095.89,4297.97,3659.17
3,BND,Illinois,40.05000,-88.36667,1997,5061.24,3513.42,4041.11,2966.04,2012.24,4978.89,3886.33,5925.26,7167.64,9868.82,5873.09,4452.61
4,BND,Illinois,40.05000,-88.36667,1998,4977.41,2695.79,1385.49,2085.67,3734.47,7184.38,6224.56,5529.68,7570.58,6596.90,5956.72,3211.31


In [157]:
#roll up data to global level by yearly and monthly
global_avg_AER = monthly_avg_AER[['Year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug',\
                                  'Sep','Oct','Nov','Dec']].groupby('Year').agg(np.mean)

global_avg_AER = global_avg_AER.round(decimals = 2)

global_avg_AER.reset_index(inplace = True)

global_avg_AER.head(20)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Year,,,,,,,,,,,,
1975,795.03,795.03,795.03,451.73,795.03,580.06,1179.10,711.68,818.22,685.29,1304.93,629.23
1976,294.08,603.32,553.45,687.52,923.89,594.80,503.14,309.01,680.04,617.39,453.96,420.80
1977,351.42,360.20,477.35,370.58,249.07,267.82,252.70,343.96,466.99,372.39,359.08,343.99
1978,365.60,304.98,369.18,316.16,311.91,328.10,292.04,471.29,278.78,211.77,289.08,291.86
1979,339.98,454.37,428.58,361.95,272.81,345.56,504.45,358.89,287.38,288.07,276.47,269.10
1980,286.70,265.14,280.23,232.85,193.63,456.52,292.32,318.44,292.24,270.09,237.37,433.11
1981,414.78,394.10,326.46,320.69,342.30,401.25,637.47,532.01,420.42,402.81,363.78,521.20
1982,464.90,409.20,401.59,280.60,244.69,402.03,393.30,363.90,354.81,233.98,243.73,280.15
1983,221.37,267.56,334.12,387.80,386.20,277.29,341.75,395.37,388.78,346.38,232.03,230.69


In [159]:
#write raw Aerosol data and summary Aerosol data to csv files
monthly_avg_AER.to_csv(os.path.join(clnd_data_dir,"AER_RawData.csv"), index = False)  ## Entire Raw DAta with stations

global_avg_AER.to_csv(os.path.join(clnd_data_dir,"AER_GlobalSummary.csv"), index = False) ## Grouped by year and reported monthly at global lvl

### ***************** Aerosol Extraction and cleaning done *************************

### Extract & clean Total Solar Irradiance Data

In [162]:
# Extract JSON though requests.get()
try:
    resp = requests.get(tsi_url)
    
    #check if the status code is other 200 (ie. not successful request)
    if(resp.status_code != 200):
        raise HTTPError
    
    # extract the JSON data
    TSI_json = resp.json()
    
except ConnectionError as c:
    print("Error in Connection :" + e)

except HTTPError as h:
    print("Unsuccessful in obtaining JSON : " + h)
    

In [163]:
## Print JSON Outout for verification
pprint(TSI_json)

{'nrl2_tsi_P1M': {'samples': [{'irradiance': 1361.3564453125,
                               'time': '1970-15-01',
                               'uncertainty': 0.5603513121604919},
                              {'irradiance': 1361.090576171875,
                               'time': '1970-46-02',
                               'uncertainty': 0.6423571705818176},
                              {'irradiance': 1361.6412353515625,
                               'time': '1970-74-03',
                               'uncertainty': 0.5571277141571045},
                              {'irradiance': 1361.3487548828125,
                               'time': '1970-105-04',
                               'uncertainty': 0.610397219657898},
                              {'irradiance': 1361.2384033203125,
                               'time': '1970-135-05',
                               'uncertainty': 0.6647059917449951},
                              {'irradiance': 1361.541015625,
                 

                               'time': '1985-196-07',
                               'uncertainty': 0.10857190191745758},
                              {'irradiance': 1360.765869140625,
                               'time': '1985-227-08',
                               'uncertainty': 0.0635213553905487},
                              {'irradiance': 1360.6533203125,
                               'time': '1985-258-09',
                               'uncertainty': 0.033385731279850006},
                              {'irradiance': 1360.544921875,
                               'time': '1985-288-10',
                               'uncertainty': 0.05904655158519745},
                              {'irradiance': 1360.608642578125,
                               'time': '1985-319-11',
                               'uncertainty': 0.04473700746893883},
                              {'irradiance': 1360.65234375,
                               'time': '1985-349-12',
                         

                               'time': '1997-166-06',
                               'uncertainty': 0.06938306987285614},
                              {'irradiance': 1360.7947998046875,
                               'time': '1997-196-07',
                               'uncertainty': 0.06440741568803787},
                              {'irradiance': 1360.8487548828125,
                               'time': '1997-227-08',
                               'uncertainty': 0.10447163879871368},
                              {'irradiance': 1360.6614990234375,
                               'time': '1997-258-09',
                               'uncertainty': 0.2156926840543747},
                              {'irradiance': 1361.0147705078125,
                               'time': '1997-288-10',
                               'uncertainty': 0.12876693904399872},
                              {'irradiance': 1360.889404296875,
                               'time': '1997-319-11',
             

                              {'irradiance': 1360.639892578125,
                               'time': '2009-135-05',
                               'uncertainty': 0.02759220451116562},
                              {'irradiance': 1360.618896484375,
                               'time': '2009-166-06',
                               'uncertainty': 0.02760385349392891},
                              {'irradiance': 1360.5867919921875,
                               'time': '2009-196-07',
                               'uncertainty': 0.0266764797270298},
                              {'irradiance': 1360.5853271484375,
                               'time': '2009-227-08',
                               'uncertainty': 0.014349465258419514},
                              {'irradiance': 1360.5870361328125,
                               'time': '2009-258-09',
                               'uncertainty': 0.023916643112897873},
                              {'irradiance': 1360.5994873046875,
 

                               'time': '2015-105-04',
                               'uncertainty': 0.39091238379478455},
                              {'irradiance': 1361.48486328125,
                               'time': '2015-135-05',
                               'uncertainty': 0.355719655752182},
                              {'irradiance': 1361.2862548828125,
                               'time': '2015-166-06',
                               'uncertainty': 0.37156999111175537},
                              {'irradiance': 1361.412353515625,
                               'time': '2015-196-07',
                               'uncertainty': 0.31693947315216064},
                              {'irradiance': 1361.0853271484375,
                               'time': '2015-227-08',
                               'uncertainty': 0.29743823409080505},
                              {'irradiance': 1361.046142578125,
                               'time': '2015-258-09',
                 

In [164]:
# Extract JSON data into dataframe using normalize function
TSI_df = json_normalize(TSI_json['nrl2_tsi_P1M']['samples'])

TSI_df.head()
              

,irradiance,time,uncertainty
0,1361.356445,1970-15-01,0.560351
1,1361.090576,1970-46-02,0.642357
2,1361.641235,1970-74-03,0.557128
3,1361.348755,1970-105-04,0.610397
4,1361.238403,1970-135-05,0.664706


In [165]:
#Write the raw data into data folder
TSI_df.to_csv(os.path.join(clnd_data_dir,"TSI_rawdata_fromJSON.csv"), index = False)

## Clean the data
 - Step 1 : Extract Year and Month from time column
 - Step 2 : reshape data so that Month is displayed across columns 
 - Step 3: Check for missing values
 - Step 4: If values are missing, replace them with mean for the year
 

In [166]:
# Step 1 : Extract Year and Month from time column
split_yr_mnth = TSI_df['time'].str.split("-")

TSI_df['Year'] = [v[0] for v in split_yr_mnth]
TSI_df['Month'] = [v[2] for v in split_yr_mnth]

TSI_df.head()

,irradiance,time,uncertainty,Year,Month
0,1361.356445,1970-15-01,0.560351,1970,01
1,1361.090576,1970-46-02,0.642357,1970,02
2,1361.641235,1970-74-03,0.557128,1970,03
3,1361.348755,1970-105-04,0.610397,1970,04
4,1361.238403,1970-135-05,0.664706,1970,05


In [167]:
#Step 2: reshape data so that Month is displayed across columns 

TSI_df_fnl = TSI_df.pivot_table(index = ['Year'],columns = 'Month', values = 'irradiance')

TSI_df_fnl.reset_index(inplace = True)

# Rename month number to names
mnth_names = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun',\
              '07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov', '12':'Dec'}

TSI_df_fnl.rename(columns = mnth_names, inplace = True)

TSI_df_fnl.rename_axis("", axis = 1, inplace = True)

TSI_df_fnl.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1970,1361.356445,1361.090576,1361.641235,1361.348755,1361.238403,1361.541016,1361.328857,1361.303101,1361.335693,1361.390015,1361.366943,1361.544800
1,1971,1361.068115,1361.079346,1361.334229,1361.090576,1361.068359,1361.143066,1360.915405,1360.887695,1361.090698,1360.843506,1360.992798,1360.974854
2,1972,1361.261719,1361.021240,1361.064453,1361.411987,1360.911255,1361.129150,1361.247925,1361.107300,1361.278687,1360.903809,1361.025879,1360.847900
3,1973,1360.994019,1360.989502,1360.784302,1360.887573,1360.996704,1360.930176,1360.851318,1360.848389,1360.680054,1360.842407,1360.782349,1360.796875
4,1974,1360.768555,1360.731689,1360.846680,1360.693481,1360.897705,1360.935059,1360.620239,1360.803101,1360.700195,1360.696045,1360.851929,1360.705933


In [168]:
# Step 3: Check for missing values
#Step 4: Replace missing values with row wise averages for months
if(TSI_df_fnl.isnull().values.sum() != 0):
    print("has missing values")
    
    #check to see if it is in the year
    if(TSI_df_fnl['Year'].isnull().values.sum() != 0):
        print('{TSI_df_fnl["Year"].isnull().values.sum()} Year(s) are missing')
    
    if(TSI_df_fnl['Month'].isnull().values.sum() != 0):
        rowAvg = TSI_df_fnl[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']].mean(axis = 1)
        # Replace NAN with row-wise mean
        TSI_df_fnl = TSI_df_fnl.apply(lambda r: r.fillna(rowAvg[r.index]))
        
else:
    print("Data has no missing values")

TSI_df_fnl = TSI_df_fnl.round(decimals = 2)
TSI_df_fnl.head()

Data has no missing values


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1970,1361.36,1361.09,1361.64,1361.35,1361.24,1361.54,1361.33,1361.30,1361.34,1361.39,1361.37,1361.54
1,1971,1361.07,1361.08,1361.33,1361.09,1361.07,1361.14,1360.92,1360.89,1361.09,1360.84,1360.99,1360.97
2,1972,1361.26,1361.02,1361.06,1361.41,1360.91,1361.13,1361.25,1361.11,1361.28,1360.90,1361.03,1360.85
3,1973,1360.99,1360.99,1360.78,1360.89,1361.00,1360.93,1360.85,1360.85,1360.68,1360.84,1360.78,1360.80
4,1974,1360.77,1360.73,1360.85,1360.69,1360.90,1360.94,1360.62,1360.80,1360.70,1360.70,1360.85,1360.71


In [169]:
# Write to CSV 
TSI_df_fnl.to_csv(os.path.join(clnd_data_dir,"TSI_MonthlyAvg.csv"), index = False)

## DATA EXTRACTION AND CLEANING DONE *****************